In [1]:
import json
import pandas as pd
from IPython.display import display, Markdown

def read_experiment_data(file_path):
    """读取实验数据文件"""
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data


def extract_experiment_info(experiments):
    """提取实验信息"""
    results = []

    for exp in experiments:
        config = exp.get("config", {})
        result = exp.get("result", {})

        # 提取所需的参数
        edge_cloud_bandwidth = config.get("edge_cloud_bandwidth", 0)
        edge_end_bandwidth = config.get("edge_end_bandwidth", 0)
        eval_mode = result.get("eval_mode", "")
        target_forward_times = result.get("target_forward_times", 0)
        wall_time = result.get("wall_time", 0)
        communication_time = result.get("communication_time", 0)
        edge_cloud_data_bytes = result.get("edge_cloud_data_bytes", 0)
        throughput = result.get("throughput", 0)
        comm_energy = result.get("comm_energy", 0)

        results.append(
            {
                "Edge-Cloud 带宽 (Mbps)": edge_cloud_bandwidth,
                "Edge-End 带宽 (Mbps)": edge_end_bandwidth,
                "方法": eval_mode,
                "目标模型 Forward 次数": target_forward_times,
                "挂钟时间 (s)": round(wall_time, 2),
                "通信时间 (s)": round(communication_time, 2),
                "Edge-Cloud 数据传输量 (bytes)": edge_cloud_data_bytes,
                "Token 输出速率 (tokens/s)": round(throughput, 2),
                "通信能量": comm_energy,
            }
        )

    return results


def generate_markdown_table(data):
    """生成 markdown 表格"""
    df = pd.DataFrame(data)

    # 按方法分组，然后按Edge-Cloud带宽降序排序
    df_sorted = df.sort_values(
        ["方法", "Edge-Cloud 带宽 (Mbps)"], ascending=[True, False]
    )

    # 生成 markdown 表格
    markdown_table = df_sorted.to_markdown(index=False, tablefmt="github")
    return markdown_table


def get_pandas_table(data):
    """生成 pandas DataFrame 表格"""
    df = pd.DataFrame(data)

    # 按方法分组，然后按Edge-Cloud带宽降序排序
    df_sorted = df.sort_values(
        ["方法", "Edge-Cloud 带宽 (Mbps)"], ascending=[True, False]
    )
    return df_sorted

def generate_grouped_tables(data):
    """按方法分组生成多个表格"""
    df = pd.DataFrame(data)

    # 按方法分组
    grouped = df.groupby("方法")

    all_tables = []

    for method, group_df in grouped:
        # 按Edge-Cloud带宽降序排序
        group_df_sorted = group_df.sort_values(
            "Edge-Cloud 带宽 (Mbps)", ascending=False
        )

        # 生成表格
        table = group_df_sorted.to_markdown(index=False, tablefmt="github")

        all_tables.append(f"## {method}\n\n{table}\n")

    return "\n".join(all_tables)


def jupyter_display(experiment_data):
    """
    一个专门为 Jupyter Notebook 设计的主函数，
    用于读取、处理数据并以美观的表格形式显示结果。
    """
    # 创建
    df = pd.DataFrame(experiment_data)
    # --- 1. 显示统一排序的表格 ---
    # 使用 display(Markdown(...)) 来渲染标题
    display(Markdown("# 实验结果对比表（按方法和带宽排序）"))

    # 对 DataFrame 进行排序
    df_sorted = df.sort_values(
        ["方法", "Edge-Cloud 带宽 (Mbps)"], ascending=[True, False]
    )

    # 直接显示 DataFrame 对象，Jupyter 会将其渲染为 HTML 表格
    display(df_sorted)
    # --- 2. 显示按方法分组的表格 ---
    display(Markdown("# 实验结果对比表（按方法分组）"))

    # 按方法进行分组
    grouped = df.groupby("方法")
    # 遍历每个分组并分别显示
    for method, group_df in grouped:
        # 显示分组标题
        display(Markdown(f"## {method}"))

        # 对当前分组的 DataFrame 进行排序
        group_df_sorted = group_df.sort_values(
            "Edge-Cloud 带宽 (Mbps)", ascending=False
        )

        # 显示排序后的分组 DataFrame
        display(group_df_sorted)


def main():
    # 读取实验数据
    file_path = "experiment_summary_20251204_203226.json"
    experiments = read_experiment_data(file_path)
    experiment_data = extract_experiment_info(experiments)
    unified_table = generate_markdown_table(experiment_data)
    grouped_tables = generate_grouped_tables(experiment_data)

    jupyter_display(experiment_data)

    # 保存到文件
    with open("experiment_results_table.md", "w", encoding="utf-8") as f:
        f.write("# 实验结果对比表（按方法和带宽排序）\n\n")
        f.write(unified_table)
        f.write("\n\n" + "=" * 80 + "\n\n")
        f.write("# 实验结果对比表（按方法分组）\n\n")
        f.write(grouped_tables)

    print("\n表格已保存到 experiment_results_table.md 文件中")


if __name__ == "__main__":
    main()

# 实验结果对比表（按方法和带宽排序）

,Edge-Cloud 带宽 (Mbps),Edge-End 带宽 (Mbps),方法,目标模型 Forward 次数,挂钟时间 (s),通信时间 (s),Edge-Cloud 数据传输量 (bytes),Token 输出速率 (tokens/s),通信能量
1,34.60,563.0,dist_spec,5328,577.64,4.04,17488858,35.45,0.0
5,34.60,563.0,dist_spec,5328,572.57,5.01,21669858,35.77,0.0
9,34.60,563.0,dist_spec,5328,575.91,5.98,25850858,35.56,0.0
13,34.60,563.0,dist_spec,5328,569.49,6.94,30031858,35.96,0.0
17,34.60,563.0,dist_spec,5328,565.99,7.91,34212858,36.18,0.0
...,...,...,...,...,...,...,...,...,...
182,0.14,33.2,uncertainty_decoding,575,8484.64,7814.26,27175044,2.41,0.0
186,0.14,33.2,uncertainty_decoding,575,8482.13,7814.26,27175044,2.41,0.0
190,0.14,33.2,uncertainty_decoding,575,8480.82,7814.26,27175044,2.41,0.0
194,0.14,33.2,uncertainty_decoding,575,8481.45,7814.26,27175044,2.41,0.0


# 实验结果对比表（按方法分组）

## dist_spec

,Edge-Cloud 带宽 (Mbps),Edge-End 带宽 (Mbps),方法,目标模型 Forward 次数,挂钟时间 (s),通信时间 (s),Edge-Cloud 数据传输量 (bytes),Token 输出速率 (tokens/s),通信能量
1,34.60,563.0,dist_spec,5328,577.64,4.04,17488858,35.45,0.0
5,34.60,563.0,dist_spec,5328,572.57,5.01,21669858,35.77,0.0
9,34.60,563.0,dist_spec,5328,575.91,5.98,25850858,35.56,0.0
13,34.60,563.0,dist_spec,5328,569.49,6.94,30031858,35.96,0.0
17,34.60,563.0,dist_spec,5328,565.99,7.91,34212858,36.18,0.0
21,34.60,563.0,dist_spec,5328,571.14,8.88,38393858,35.86,0.0
25,34.60,563.0,dist_spec,5328,574.97,9.84,42574858,35.62,0.0
29,34.60,563.0,dist_spec,5328,578.71,10.81,46755858,35.39,0.0
33,34.60,563.0,dist_spec,5328,577.46,11.78,50936858,35.47,0.0
37,34.60,563.0,dist_spec,5328,578.22,12.74,55117858,35.42,0.0


...

## tridecoding

,Edge-Cloud 带宽 (Mbps),Edge-End 带宽 (Mbps),方法,目标模型 Forward 次数,挂钟时间 (s),通信时间 (s),Edge-Cloud 数据传输量 (bytes),Token 输出速率 (tokens/s),通信能量
3,34.60,563.0,tridecoding,4604,856.65,0.32,1290380,24.87,0.0
7,34.60,563.0,tridecoding,4604,814.88,0.46,1843380,26.14,0.0
11,34.60,563.0,tridecoding,4604,819.10,0.60,2396380,26.01,0.0
15,34.60,563.0,tridecoding,4604,824.29,0.74,2949380,25.84,0.0
19,34.60,563.0,tridecoding,4604,818.86,0.88,3502380,26.01,0.0
23,34.60,563.0,tridecoding,4604,823.06,1.02,4055380,25.88,0.0
27,34.60,563.0,tridecoding,4604,826.41,1.16,4608380,25.78,0.0
31,34.60,563.0,tridecoding,4604,815.92,1.31,5161380,26.11,0.0
35,34.60,563.0,tridecoding,4604,816.11,1.45,5714380,26.10,0.0
39,34.60,563.0,tridecoding,4604,821.44,1.59,6267380,25.93,0.0


## uncertainty_decoding

,Edge-Cloud 带宽 (Mbps),Edge-End 带宽 (Mbps),方法,目标模型 Forward 次数,挂钟时间 (s),通信时间 (s),Edge-Cloud 数据传输量 (bytes),Token 输出速率 (tokens/s),通信能量
2,34.60,563.0,uncertainty_decoding,575,6936.06,6267.68,27175044,2.95,0.0
6,34.60,563.0,uncertainty_decoding,575,6939.28,6267.68,27175044,2.95,0.0
10,34.60,563.0,uncertainty_decoding,575,6937.73,6267.68,27175044,2.95,0.0
14,34.60,563.0,uncertainty_decoding,575,6934.66,6267.68,27175044,2.95,0.0
18,34.60,563.0,uncertainty_decoding,575,6937.10,6267.68,27175044,2.95,0.0
22,34.60,563.0,uncertainty_decoding,575,6934.46,6267.68,27175044,2.95,0.0
26,34.60,563.0,uncertainty_decoding,575,6937.80,6267.68,27175044,2.95,0.0
30,34.60,563.0,uncertainty_decoding,575,6934.80,6267.68,27175044,2.95,0.0
34,34.60,563.0,uncertainty_decoding,575,6936.16,6267.68,27175044,2.95,0.0
38,34.60,563.0,uncertainty_decoding,575,6937.69,6267.68,27175044,2.95,0.0



表格已保存到 experiment_results_table.md 文件中
